# Parsing data

In [ ]:
from __future__ import print_function

In [ ]:
import re
import os
import glob

In [ ]:
scrape_dir = os.path.join('..', 'data-scrapes')
print(scrape_dir)

In [ ]:
import datetime, time
ts = time.time()
st = datetime.datetime.fromtimestamp(ts).strftime('%Y%m%d-%H%M%S')

out_file = os.path.join('..', 'data', 'protein-seqs_' + st + '.txt')
print("Converting sequences. . . to [" +out_file+"]")

In [ ]:
num_proteins_done = 0

fasta_files = glob.glob(scrape_dir + "/*fasta")
print(fasta_files)

In [ ]:
def dump_to_file(protein_id, seq):
    print('Writing', protein_id, seq, "to file", out_file)
    with open(out_file, "a") as f:
        f.write(protein_id + "," + seq + "\n")

In [ ]:
for fname in fasta_files:
    print("Converting %s:" % fname)
    
    with open(fname, 'r') as f:
        protein_seq = ''
        protein_id = ''
        
        for line in f:
            print('Processing line', num_proteins_done)
            match = re.search(r'^>([a-z]{2})\|([A-Z0-9]*)\|', line)
            if match:
                if protein_id != '':
                    dump_to_file(protein_id, protein_seq)
                
                num_proteins_done += 1
                if num_proteins_done > 10: break
                
                protein_id = match.group(2)
                protein_seq = ''
            else:
                protein_seq += line.strip()
        if protein_id != '':
            dump_to_file(protein_id, protein_seq)

In [ ]:
out_file_fns = os.path.join('..', 'data', 'protein_fun_' + st + '.txt')
print("Converting functions to [", out_file_fns, "]")

target_functions = ['0005524']

In [ ]:
annot_files = glob.glob(scrape_dir + "/*annotations.txt")
print(annot_files)

In [ ]:
func_list = []

for fname in annot_files:
    print("Parsing", fname)
    with open(fname, 'r') as f:
        for line in f:
            match = re.search(r'([A-Z0-9]*)\sGO:(.*);\sF:.*;', line)
            if match:
                protein_id = match.group(1)
                function = match.group(2)
                
                if function not in target_functions:
                    continue
                
                func_list.append(protein_id)
    import json
    with open(out_file_fns, 'w') as fp:
        json.dump(func_list, fp)
        
    print(func_list[:10])